# Debug ellipse angle loss

In [1]:
import os
import sys
import math

import numpy
as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
lib_path = os.path.join('../src/lib')
if not lib_path in sys.path:
    sys.path.append(lib_path)
    
lib_path = os.path.join('../src/')
if not lib_path in sys.path:
    sys.path.append(lib_path)
    
from models.losses import RegL1Loss4Angle

In [25]:
a1 = 180 / 180 * math.pi
a2 = -90 / 180 * math.pi

pred_angle = torch.tensor(a1)
true_angle = torch.tensor(a2)
diff_angle = pred_angle - true_angle
sin, cos = torch.sin(diff_angle), torch.cos(diff_angle)
mask = torch.ones_like(cos)
mask(cos < 0) = 1
loss = torch.atan2(sin, cos) if cos >=0 else torch.atan2(-sin, -cos)
print(diff_angle, sin, cos, loss)

tensor(4.7124) tensor(-1.) tensor(1.1925e-08) tensor(-1.5708)


In [98]:
def angle_loss(pred_angle, true_angle, size_average=False):
    diff_angle = (pred_angle - true_angle) * math.pi
    sin, cos = torch.sin(diff_angle), torch.cos(diff_angle)
    mask = torch.ones_like(cos)
    mask[cos < 0] = -1
    sin, cos = sin * mask, cos * mask
    loss = F.l1_loss(torch.atan2(sin, cos), torch.zeros_like(sin), size_average=False) #if cos >=0 else torch.atan2(-sin, -cos)
    return loss

In [72]:
pred_angle = torch.tensor(0.4)
true_angle = torch.tensor(1)
diff_angle = (pred_angle - true_angle) * math.pi
angle_loss(pred_angle, true_angle)

tensor(1.2566)

In [89]:
np.mean(np.abs([1.2566e+00,  1.2566e+00, -1.5708e+00, -8.7423e-08, -1.5708e+00, 1.5708e+00]))

1.2042666812371667

In [4]:
pred_angle = torch.tensor([1,    1,    2,  0,  1, 0.5])
true_angle = torch.tensor([-0.4, 0.6, 0.5, 1, 0.5, 1])
diff_angle = (pred_angle - true_angle) * math.pi
sin, cos = torch.sin(diff_angle), torch.cos(diff_angle)
angle_loss(pred_angle, true_angle)
# tensor([ 1.2566e+00,  1.2566e+00, -1.5708e+00, -8.7423e-08, -1.5708e+00, 1.5708e+00])

In [31]:
def _gather_feat(feat, ind, mask=None):
    dim  = feat.size(2)
    ind  = ind.unsqueeze(2).expand(ind.size(0), ind.size(1), dim)
    feat = feat.gather(1, ind)
    if mask is not None:
        mask = mask.unsqueeze(2).expand_as(feat)
        feat = feat[mask]
        feat = feat.view(-1, dim)
    return feat

def _transpose_and_gather_feat(feat, ind):
    feat = feat.permute(0, 2, 3, 1).contiguous()
    feat = feat.view(feat.size(0), -1, feat.size(3))
    feat = _gather_feat(feat, ind)
    return feat

In [6]:
ind = torch.tensor(np.random.randint(0, 10, [16, 2])) 
mask = torch.tensor(np.ones([16, 2])) 
target = torch.tensor(np.ones([16, 2, 1])) * 0.5 
feat = torch.tensor(np.zeros([16, 1, 128, 128]))
print(feat.shape, mask.shape, ind.shape, target.shape)
# feat = feat.permute(0, 2, 3, 1).contiguous() # (16, 128, 128, 1)
# feat = feat.view(feat.size(0), -1, feat.size(3)) # （16, 16384, 1）
# feat = _gather_feat(feat, ind) # (16, 2, 1)
# feat.shape

torch.Size([16, 1, 128, 128]) torch.Size([16, 2]) torch.Size([16, 2]) torch.Size([16, 2, 1])


In [100]:
class RegL1Loss(nn.Module):
  def __init__(self):
    super(RegL1Loss, self).__init__()
  
  def forward(self, output, mask, ind, target):
    pred = _transpose_and_gather_feat(output, ind)
    mask = mask.unsqueeze(2).expand_as(pred).float()
    # loss = F.l1_loss(pred * mask, target * mask, reduction='elementwise_mean')
    loss = F.l1_loss(pred * mask, target * mask, size_average=False)
    loss = loss / (mask.sum() + 1e-4)
    return loss

class RegL1Loss4Angle(nn.Module):
  def __init__(self):
    super(RegL1Loss4Angle, self).__init__()
  
  def forward(self, output, mask, ind, target):
    pred = _transpose_and_gather_feat(output, ind)
    mask = mask.unsqueeze(2).expand_as(pred).float()
    # loss = F.l1_loss(pred * mask, target * mask, reduction='elementwise_mean')
    loss = angle_loss(pred * mask, target * mask, size_average=False)
    loss = loss / (mask.sum() + 1e-4)
    return loss

In [65]:
%pdb off

Automatic pdb calling has been turned OFF


In [8]:
RegL1Loss4Angle(True)(feat, mask, ind, target)

tensor(1.0708, dtype=torch.float64)

In [37]:
feat = torch.tensor(np.ndarray([16, 2, 128, 128]))
mask = torch.tensor(np.zeros([16, 1, 128, 128]))
feat.shape, mask.shape

(torch.Size([16, 2, 128, 128]), torch.Size([16, 1, 128, 128]))

In [ ]:
class RegL1Loss(nn.Module):
  def __init__(self):
    super(RegL1Loss, self).__init__()
  
  def forward(self, output, mask, ind, target):
    pred = _transpose_and_gather_feat(output, ind)
    mask = mask.unsqueeze(2).expand_as(pred).float()
    # loss = F.l1_loss(pred * mask, target * mask, reduction='elementwise_mean')
    loss = F.l1_loss(pred * mask, target * mask, size_average=False)
    loss = loss / (mask.sum() + 1e-4)
    return loss